1. Tìm danh sách gợi ý phim

In [1]:
"""
Tạo một dataset như mô tả. Dataset là kiểu dict với mỗi phần tử có key là tên user
và value là dict của các phim (key) và đánh giá của nó (value)
"""
#Dataset dạng: {người-dùng1: {phim1:đánh-giá,phim2:đánh giá...},...}
dataset={
    'Lisa Rose': {'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'Superman Returns': 3.5,
        'You, Me and Dupree': 2.5,
        'The Night Listener': 3.0},
    'Gene Seymour': {'Lady in the Water': 3.0,
        'Snakes on a Plane': 3.5,
        'Just My Luck': 1.5,
        'Superman Returns': 5.0,
        'The Night Returns': 3.0,
        'You, Me and Dupree': 3.5},
    'Michael Phillips': {'Lady in the Water': 2.5,
        'Snakes on a Plane': 3.0,
        'Superman Returns': 3.5,
        'The Night Listener': 4.0},
    'Claudia Puig': {'Snakes on a Plane': 3.5,
        'Just My Luck': 3.0,
        'Superman Returns': 4.0,
        'The Night Listener': 4.0,
        'You, Me and Dupree': 2.5},
    'Mick LaSalle': {'Lady in the Water': 3.0,
        'Snakes on a Plane': 4.0,
        'Just My Luck': 2.0,
        'Superman Returns': 3.0,
        'The Night Listener': 3.0,
        'You, Me and Dupree': 2.0},
    'Jack Matthews': {'Lady in the Water': 3.0,
        'Snakes on a Plane': 40,
        'The Night Listener': 3.0,
        'Superman Returns': 5.0,
        'You, Me and Dupree': 3.5},
    'Toby': {'Snakes on a Plane': 4.5,
        'You, Me and Dupree': 1.0,
        'Superman Returns': 4.0}
}


Từ tập dữ liệu, trích danh sách người dùng và danh sách item, lưu danh sách người dùng vào tập user_set và danh sách các item vào tập item_set

In [5]:
#tập người dùng
user_set = list(dataset.keys())
#tập item
item_set = []
for user, items in dataset.items():
    item_set = item_set + list(items.keys())
item_set = list(set(item_set))
print(user_set)
print(item_set)

['Lisa Rose', 'Gene Seymour', 'Michael Phillips', 'Claudia Puig', 'Mick LaSalle', 'Jack Matthews', 'Toby']
['You, Me and Dupree', 'Just My Luck', 'Lady in the Water', 'Snakes on a Plane', 'Superman Returns', 'The Night Returns', 'The Night Listener']


In [6]:
#Hàm tính trung bình đánh giá của người dùng
def cal_mean_rating_user(dataset):
    mean_user = {}
    for u, items in dataset.items():
        num_rating = len(items)
        sum_rating = 0
        for i in dataset[u]:
            sum_rating += dataset[u][i]
        mean_user[u] = sum_rating/num_rating
    return mean_user

In [8]:
#Tính hệ số tương quan giữa 2 người dùng
from math import sqrt

def pearson_correlation(dataset, user1, user2, mean_user):
    both_rated = {}
    for item in dataset[user1]:
        if item in dataset[user2]:
            both_rated[item] = 1
    mean_rating_user_1 = [(dataset[user1][item] - mean_user[user1]) for item in both_rated ]
    mean_rating_user_2 = [(dataset[user2][item] - mean_user[user2]) for item in both_rated ]

    tu_so = sum([i * j for i, j in zip(mean_rating_user_1, mean_rating_user_2)])
    mau_so_user_1 = sum([pow(i,2) for i in mean_rating_user_1])
    mau_so_user_1 = sqrt(mau_so_user_1)
    mau_so_user_2 = sum([pow(i,2) for i in mean_rating_user_2])
    mau_so_user_2 = sqrt(mau_so_user_2)
    if mau_so_user_1 * mau_so_user_2 == 0:
        return 0
    return tu_so/(mau_so_user_1 * mau_so_user_2)


In [9]:
#Tính ma trận tương quan theo Pearson giữa các người dùng
def cal_pearson_sim(dataset, mean_user):
    sim_pearson = {}
    users = list(dataset.keys())
    for user in users:
        sim_pearson[user] ={}

    for i, user in enumerate(users):
        for j in range(i, len(users)):
            if user == users[j]: continue

            sim = pearson_correlation(dataset, user, users[j], mean_user)
            sim_pearson[user][users[j]] = sim
            sim_pearson[users[j]][user] = sim
        return sim_pearson 

In [10]:
#Dự đoán rating cho các sản phẩm chưa được đánh giá bởi người dùng đó
def predict_rating(dataset, item_set, user, mean_user, sim_matrix, top_k = None):
    sim_user = sim_matrix[user]
    if top_k !=None:
        sim_user = dict(sorted(sim_user.items(),
                               key = lambda item: (item[1], item[0]), reverse = True) [:top_k])
    rating_item = {}
    for item in item_set:
        if item not in dataset[user]:
            tu = 0; mau = 0
            for u in sim_user:
                if item in dataset[u]:
                    tu += sim_user[u] * (dataset[u][item] - mean_user[u])
                    mau += abs(sim_user[u])
            if mau == 0:
                rating_item[item] = 0
            else:
                rating_item[item] = mean_user[user] + tu/mau
    if len(rating_item):
        rating_item = dict(sorted(rating_item.items(),
                           key = lambda item:item[1],
                           reverse=True))
    return rating_item

In [11]:
#Quá trình dự đoán các đánh giá của người dùng được thực hiện
mean_user = cal_mean_rating_user(dataset)

sim = cal_pearson_sim(dataset, mean_user)
result =predict_rating(dataset, item_set, 'Toby', mean_user, sim)
print(result)
result = predict_rating(dataset, item_set, 'Jack Matthews', mean_user,sim)
print(result)
result = predict_rating(dataset, item_set, 'Michael Phillips', mean_user, sim )
print(result)
result = predict_rating(dataset, item_set, 'Claudia Puig', mean_user, sim)
print(result)

{'Just My Luck': 3.1666666666666665, 'The Night Listener': 3.1666666666666665, 'Lady in the Water': 2.6666666666666665, 'The Night Returns': 0}
{'Just My Luck': 10.9, 'The Night Returns': 0}
{'Just My Luck': 3.25, 'You, Me and Dupree': 2.75, 'The Night Returns': 0}
{'Lady in the Water': 2.9, 'The Night Returns': 0}


In [ ]:
import os  
from math import sqrt  
import sys  
import csv  

class Collaborative_Filter:
    # Định nghĩa lớp chính cho bộ lọc cộng tác
    def __init__(self, input_file_name, user_id, movie, k):
        # Hàm khởi tạo (constructor) của lớp
        self.input_file_name = input_file_name  # Tên file dữ liệu đầu vào (ví dụ: 'ratings.csv')
        self.user_id = user_id  # ID của user cần dự đoán/gợi ý
        self.movie = movie  # Tên/ID của phim cần dự đoán rating
        self.k = k  # Số lượng hàng xóm (neighbors) gần nhất sẽ sử dụng
        self.dataset = None  # Sẽ chứa toàn bộ dữ liệu thô (danh sách các hàng)
        self.uu_dataset = None  # Sẽ chứa dữ liệu user-user (dạng dict: {userId: {movieId: rating}})
        self.ii_dataset = None  # Sẽ chứa dữ liệu item-item (dạng dict: {movieId: {userId: rating}})

    def load_data(self, input_file_name):
        # Hàm để tải dữ liệu từ file .csv
        dataset = []  # Khởi tạo danh sách lưu trữ toàn bộ dataset
        uu_dataset = {}  # Khởi tạo dictionary cho user-user
        ii_dataset = {}  # Khởi tạo dictionary cho item-item

        # Sử dụng 'with open' để đảm bảo file được đóng đúng cách sau khi đọc
        # 'encoding='utf-8'' giúp đọc các ký tự đặc biệt (ví dụ: Tiếng Việt)
        with open(input_file_name, 'r', encoding='utf-8') as input_file:
            # Sử dụng module csv để đọc file, chỉ định dấu phẩy là ký tự phân cách
            csv_reader = csv.reader(input_file, delimiter=',')
            
            # Bỏ qua hàng tiêu đề (header) của file csv (ví dụ: "userId,movieId,rating")
            next(csv_reader, None)
            
            # Lặp qua từng hàng (row) trong file csv
            for row in csv_reader:
                # Nếu hàng bị rỗng (ví dụ: dòng cuối file), bỏ qua
                if not row:
                    continue
                user_id_str, rating_str, movie_id_str = row[0], row[1], row[2]
                
                # Thêm hàng gốc (dạng list) vào danh sách dataset
                dataset.append(row)

                # --- Xây dựng cấu trúc user-user: {user_id: {movie_id: rating}} ---
                # Nếu user_id chưa có trong uu_dataset, tạo một dict rỗng cho user đó
                uu_dataset.setdefault(user_id_str, {})
                # Thêm rating của movie cho user đó. Chuyển rating sang dạng số thực (float)
                uu_dataset[user_id_str][movie_id_str] = float(rating_str)

                # --- Xây dựng cấu trúc item-item: {movie_id: {user_id: rating}} ---
                # Nếu movie_id chưa có trong ii_dataset, tạo một dict rỗng cho movie đó
                ii_dataset.setdefault(movie_id_str, {})
                # Thêm rating của user cho movie đó
                ii_dataset[movie_id_str][user_id_str] = float(rating_str)

        # Trả về 3 cấu trúc dữ liệu đã xử lý
        return dataset, uu_dataset, ii_dataset

    def quit(self, message):
        # Hàm trợ giúp để in thông báo lỗi và thoát chương trình
        print(message)  # In thông báo lỗi
        sys.exit(1)  # Thoát chương trình với mã lỗi 1 (báo hiệu có lỗi)

    def initialize(self):
        # Hàm khởi tạo, kiểm tra file và tải dữ liệu
        if not os.path.isfile(self.input_file_name):
            # Kiểm tra xem file dữ liệu (đã cung cấp) có tồn tại không
            self.quit('Khong tim thay file')  # Nếu không, gọi hàm quit để thoát
        
        # Tải dữ liệu từ file vào các biến của class
        self.dataset, self.uu_dataset, self.ii_dataset = self.load_data(self.input_file_name)

        # Lấy danh sách tất cả user IDs từ uu_dataset
        users = self.uu_dataset.keys()
        
        # Kiểm tra xem user_id đầu vào (cần dự đoán) có trong dataset không
        if self.user_id not in users:
            self.quit('User ID khong ton tai')  # Nếu không, thoát

    def user_average_rating(self, user_data):
        # Tính rating trung bình của một user cụ thể
        # user_data là một dict dạng {movieId: rating}
        avg_rating = 0  # Khởi tạo tổng rating
        size = len(user_data)  # Lấy số lượng phim mà user đã rate
        
        # Lặp qua từng (phim, rating) trong dữ liệu của user
        for (movie, rating) in user_data.items():
            avg_rating += float(rating)  # Cộng dồn rating
            
        # Chia để lấy trung bình, nhân 1.0 để đảm bảo kết quả là số thực
        avg_rating /= size * 1.0
        return avg_rating  # Trả về rating trung bình

    def common_items(self, user1_data, user2_data):
        # Tìm các item (phim) chung mà 2 user cùng rate
        result = []  # Khởi tạo danh sách item chung
        ht = {}  # Khởi tạo một hash table (dictionary) để đếm
        
        # Lặp qua các phim của user 1
        for (movie, rating) in user1_data.items():
            ht.setdefault(movie, 0)  # Đặt giá trị mặc định là 0 nếu phim chưa có trong ht
            ht[movie] += 1  # Tăng bộ đếm cho phim này lên 1
        
        # Lặp qua các phim của user 2
        for (movie, rating) in user2_data.items():
            ht.setdefault(movie, 0)  # Đặt giá trị mặc định là 0
            ht[movie] += 1  # Tăng bộ đếm cho phim này lên 1
            
        # Lặp qua hash table đã đếm
        for (k, v) in ht.items():
            if v == 2:
                # Nếu một phim có bộ đếm là 2
                result.append(k)  # Nghĩa là cả 2 user đều đã rate, thêm vào danh sách chung
        
        return result  # Trả về danh sách các phim chung

    def pearson_correlation(self, user1, user2):
        # Tính độ tương đồng Pearson (Pearson Correlation Coefficient) giữa 2 user
        result = 0.0  # Khởi tạo kết quả
        user1_data = self.uu_dataset[user1]  # Lấy dữ liệu rating của user 1
        
        user2_data = self.uu_dataset[user2] #lấy dữ liệu rating của user 2 từ uu_dataset
        
        r_user1_avg = self.user_average_rating(user1_data)  # Tính rating trung bình của user 1
        r_user2_avg = self.user_average_rating(user2_data)  # Tính rating trung bình của user 2
        
        # Lấy danh sách các phim mà cả 2 user cùng rate
        item_common = self.common_items(user1_data, user2_data)

        # Nếu không có phim chung, độ tương đồng là 0
        if len(item_common) == 0:
            return 0.0

        tu_so = 0.0  # Khởi tạo tử số (numerator) của công thức Pearson
        mau_so_trai = 0.0  # Khởi tạo vế trái của mẫu số (denominator) (cho user 1)
        mau_so_phai = 0.0  # Khởi tạo vế phải của mẫu số (cho user 2)

        # Lặp qua các phim chung
        for item in item_common:
            r_user1 = user1_data[item]  # Lấy rating của user 1 cho phim này
            r_user2 = user2_data[item]  # Lấy rating của user 2 cho phim này
            
            # Tính toán theo công thức Pearson
            tu_so += (r_user1 - r_user1_avg) * (r_user2 - r_user2_avg)  # Cộng dồn tử số
            mau_so_trai += pow((r_user1 - r_user1_avg), 2)  # Cộng dồn bình phương vế trái mẫu
            mau_so_phai += pow((r_user2 - r_user2_avg), 2)  # Cộng dồn bình phương vế phải mẫu

        mau_so_trai = sqrt(mau_so_trai)  # Lấy căn bậc hai vế trái
        mau_so_phai = sqrt(mau_so_phai)  # Lấy căn bậc hai vế phải
        
        mau_so_tong = mau_so_trai * mau_so_phai  # Mẫu số chung

        # Xử lý trường hợp chia cho 0 (khi rating của user không đổi, ví dụ: user nào đó rate 5* cho mọi phim)
        if mau_so_tong == 0:
            return 0.0  # Nếu mẫu bằng 0, độ tương đồng bằng 0
            
        result = tu_so / mau_so_tong  # Tính kết quả cuối cùng
        return result  # Trả về độ tương đồng Pearson

    def k_nearest_neighbor(self, user, k):
        # Tìm k hàng xóm gần nhất (k-nearest neighbors - KNN)
        neighbors = []  # Khởi tạo danh sách lưu (user_id, độ tương đồng)
        result = []  # Khởi tạo danh sách kết quả (k hàng xóm)
        
        # Lặp qua tất cả user trong dataset
        for (user_id, data) in self.uu_dataset.items():
            if user_id == user:
                # Nếu là chính user đang xét
                continue  # Bỏ qua, không so sánh với chính mình
                
            # Tính độ tương đồng Pearson giữa user mục tiêu và user hiện tại
            sim = self.pearson_correlation(user, user_id)
            
            # Thêm (user_id, sim) vào danh sách
            neighbors.append([user_id, sim])

        # Sắp xếp danh sách hàng xóm
        # ưu tiên 1: độ tương đồng (sim) giảm dần (neighbors[1])
        # ưu tiên 2: user_id (neighbors[0]) (chỉ để đảm bảo thứ tự sắp xếp ổn định, không bắt buộc)
        sorted_neighbors = sorted(neighbors, key=lambda neighbors: (neighbors[1], neighbors[0]), reverse=True)
        
        # Lặp k lần để lấy k hàng xóm đầu tiên
        for i in range(k):
            if i >= len(sorted_neighbors):
                # Kiểm tra nếu số hàng xóm thực tế ít hơn k (ví dụ: dataset nhỏ)
                break  # Thoát vòng lặp
            result.append(sorted_neighbors[i])  # Thêm hàng xóm vào danh sách kết quả
            
        return result  # Trả về k hàng xóm gần nhất

    def check_valid_neighbors(self, item, k_nearest_neighbors):
        # Kiểm tra xem trong k hàng xóm, những ai đã rate 'item' (phim) này
        result = []  # Khởi tạo danh sách hàng xóm hợp lệ
        
        # Lặp qua k hàng xóm đã tìm được
        for neighbor in k_nearest_neighbors:
            neighbor_id = neighbor[0]  # Lấy user_id của hàng xóm
            # Kiểm tra xem 'item' có trong danh sách phim mà hàng xóm này đã rate không
            if item in self.uu_dataset[neighbor_id].keys():  # <<< SỬA LỖI: .keys()
                result.append(neighbor)  # Nếu có, đây là hàng xóm hợp lệ để dự đoán
                
        return result  # Trả về danh sách hàng xóm hợp lệ

    def predict(self, user, item, k_nearest_neighbors):
        # Dự đoán rating của 'user' cho 'item' dựa trên k hàng xóm
        
        # Lọc ra những hàng xóm đã thực sự rate 'item' này
        valid_neighbors = self.check_valid_neighbors(item, k_nearest_neighbors)

        if not len(valid_neighbors):
            # Nếu không có hàng xóm nào hợp lệ (không ai rate phim này)
            # Trả về rating trung bình của user (một cách dự đoán cơ bản)
            return self.user_average_rating(self.uu_dataset[user])
            
        tu_so = 0.0  # Khởi tạo tử số cho công thức dự đoán (weighted average)
        mau_so = 0.0  # Khởi tạo mẫu số cho công thức dự đoán
        
        # Lặp qua các hàng xóm hợp lệ
        for neighbor in valid_neighbors:
            neighbor_id = neighbor[0]  # Lấy ID hàng xóm
            neighbor_sim = neighbor[1]  # Lấy độ tương đồng (sim) của hàng xóm
            
            # Lấy rating của hàng xóm cho 'item', trừ đi rating trung bình của hàng xóm đó
            # Đây là "độ lệch" (deviation) so với mức trung bình của hàng xóm
            rating_deviation = self.uu_dataset[neighbor_id][item] - self.user_average_rating(self.uu_dataset[neighbor_id])
            
            # Tử số: tổng của (độ tương đồng * độ lệch rating)
            tu_so += neighbor_sim * rating_deviation
            
            # Mẫu số: tổng của giá trị tuyệt đối của độ tương đồng
            mau_so += abs(neighbor_sim)

        # Xử lý trường hợp chia cho 0 (ví dụ: tất cả sim đều là 0)
        if mau_so == 0:
            return self.user_average_rating(self.uu_dataset[user])

        # Công thức dự đoán cuối cùng:
        # (rating trung bình của user) + (tổng trọng số sim * độ lệch) / (tổng |sim|)
        result = self.user_average_rating(self.uu_dataset[user]) + (tu_so / mau_so)
        
        # Giới hạn kết quả trong khoảng rating hợp lệ (ví dụ: 0.5 đến 5)
        if result > 5.0:
            result = 5.0
        elif result < 0.5:
            result = 0.5

        return result  # Trả về rating dự đoán

    def print_result(self, k_nearest_neighbors, prediction):
        # In kết quả ra màn hình 
        for neighbor in k_nearest_neighbors:
            # In ID hàng xóm và độ tương đồng
            print(neighbor[0], neighbor[1])

        print('Du doan rating la:', end='')  # In thông báo dự đoán
        print(prediction)  # In rating dự đoán

    def get_recommend(self, k, k_nearest_neighbor, user):
        # Lấy k phim đề xuất (recommend) cho user
        
        # Lấy danh sách tất cả các phim có trong dataset
        item_set = list(self.ii_dataset.keys())
        
        # Khởi tạo dict lưu {item: predicted_rating}
        rating_item = {}

        # Lặp qua tất cả các phim
        for item in item_set:
            if item in self.uu_dataset[user]:
                # Nếu user đã rate phim này rồi
                continue  # Bỏ qua, không đề xuất lại
            
            # Dự đoán rating của user cho phim này (phim user chưa xem)
            rating = self.predict(user, item, k_nearest_neighbor)
            
            # Lưu lại rating dự đoán
            rating_item[item] = rating

        # Sắp xếp các phim theo rating dự đoán giảm dần
        rating_item = dict(sorted(rating_item.items(),
                                  key=lambda rt: (rt[1], rt[0]),  # Sắp xếp theo rating (rt[1])
                                  reverse=True))  # Giảm dần
                                  
        # Trả về k phim có rating dự đoán cao nhất
        return list(rating_item.items())[:k]